In [1]:
!pip install pennylane
!pip install aiohttp fsspec h5py

In [2]:
import pennylane as qml

data = qml.data.load("other", name="plus-minus")[0]
print("Atributos disponíveis no dataset:", dir(data))
print("Campos disponíveis:", getattr(data, "fields", "Sem campos"))

Atributos disponíveis no dataset: ['img_test', 'img_train', 'labels_test', 'labels_train']
Campos disponíveis: {}


In [1]:
import pennylane as qml
from pennylane import numpy as np

data = qml.data.load("other", name="plus-minus")[0]
X = np.array(data.img_train)
Y = np.array(data.labels_train)

# Use apenas as 10 primeiras amostras para exemplo
X_train, Y_train = X[:10], Y[:10]

n_qubits = X_train.shape[1]

dev = qml.device("default.qubit", wires=n_qubits)
@qml.qnode(dev)
def circuit(x, weights):
    for i in range(n_qubits):
        qml.RY(x[i], wires=i)
    for i in range(n_qubits):
        qml.Rot(*weights[i], wires=i)
    # Retorne apenas a expectativa do primeiro qubit
    return qml.expval(qml.PauliZ(0))

def cost(weights, X, Y):
    preds = np.array([circuit(x, weights) for x in X])
    return np.mean((preds - Y) ** 2)

# Inicialização dos pesos
weights = np.random.randn(n_qubits, 3, requires_grad=True)

opt = qml.GradientDescentOptimizer(stepsize=0.2)
steps = 30

for i in range(steps):
    weights = opt.step(lambda w: cost(w, X_train, Y_train), weights)
    if i % 5 == 0:
        print(f"Step {i}: Cost = {cost(weights, X_train, Y_train):.4f}")

print("Treinamento finalizado.")

ValueError: operands could not be broadcast together with shapes (10,16) (10,) 